In [55]:
import numpy as np 
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
import  re

In [56]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [57]:
#import os
#from google.colab import drive
#drive.mount('/content/drive')

In [58]:
df = pd.read_csv('../data/input/tb_redrakor.csv', index_col=None)

In [59]:
df.head()

,rank,judul,tahun,episode,network,umur,sinopsis,genre,tags,rating,funfact
0,1,Move to Heaven,2021,10,Netflix,18+,Menceritakan kisah dari Han Geu-Roo yang mende...,"Life, Drama, Family","Autism, Father-Son Relationship, Uncle-Nephew ...",9.2,Drama ini diadaptasi dari kisah nyata yaitu So...
1,2,Hospital Playlist,2020,12,"tvN, Netflix",15+,Kisah-kisah tentang orang-orang yang menjalani...,"Friendship, Romance, Life, Medical","Strong Friendship, Doctor, Multiple Mains, Slo...",9.1,Nama para tokoh Hospital Playlist diambil dari...
2,3,Flower of Evil,2020,16,"tvN, Netflix, Vidio, iQIYI, viu, Amazon Prime ...",15+,Meskipun Baek Hee Sung menyembunyikan rahasia ...,"Thriller, Romance, Crime, Melodrama","Married Couple, Deception, Suspense, Family Se...",9.1,Judul drama ini terinspirasi dari buku puisi C...
3,4,My Mister,2018,16,"tvN, Netflix, Vidio, iQIYI, Viu",15+,Park Dong Hoon adalah seorang insinyur paruh b...,"Business, Psychological, Life, Drama, Family","Nice Male Lead, Strong Female Lead, Hardship, ...",9.1,Penggemar IU (pemeran wanita utama) merasa keb...
4,5,Prison Playbook,2017,16,"tvN, Netflix, Disney+, Viu, Vidio, iQIYI, Ama...",15+,Menceritakan kisah atlet baseball terkenal ber...,"Comedy, Life, Drama","Prison, Bromance, Wrongfully Accused, Life Les...",9.1,Latar tempat utama di drama ini adalah penjara...


In [60]:
df = df.drop(['network', 'umur', 'sinopsis', 'tags', 'funfact'], axis=1)

In [61]:
df_features = pd.concat([df['genre'].str.strip().str.get_dummies(sep=','), df[['rating']],df[['tahun']],df[['episode']]],axis=1)

df['judul'] = df['judul'].map(lambda judul:re.sub('[^A-Za-z0-9]+',' ', judul))
df_features.head()

,Business,Comedy,Crime,D,Dr,Drama,Fa,Famil,Family,Fant,...,Military,Music,Mystery,Psychological,Romance,Suspense,Thriller,rating,tahun,episode
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,9.2,2021,10
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9.1,2020,12
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.1,2020,16
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,9.1,2018,16
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,9.1,2017,16


In [62]:
df_features.columns

Index(['  Business', '  Comedy', '  Crime', '  D', '  Dr', '  Drama', '  Fa',
       '  Famil', '  Family', '  Fant', '  Fantasy', '  Friendship',
       '  Historical', '  Horror', '  Invest', '  Investigation', '  Law',
       '  Life', '  M', '  Martial Arts', '  Me', '  Medica', '  Medical',
       '  Melo', '  Melodrama', '  Military', '  Myster', '  Mystery', '  Pol',
       '  Polit', '  Political', '  Psychological', '  Romance', '  S',
       '  School', '  Sci-Fi', '  Sports', '  Sup', '  Super', '  Supernat',
       '  Supernatu', '  Supernatural', '  Suspense', '  Thriller', '  Y',
       '  Yo', '  Youth', '  Zombies', 'Action', 'Business', 'Comedy', 'Drama',
       'Food', 'Friendship', 'Historical', 'Horror', 'Life', 'Military',
       'Music', 'Mystery', 'Psychological', 'Romance', 'Suspense', 'Thriller',
       'rating', 'tahun', 'episode'],
      dtype='object')

In [63]:
from sklearn.preprocessing import MinMaxScaler

In [64]:
min_max_scaler = MinMaxScaler()
df_features = min_max_scaler.fit_transform(df_features)

In [65]:
np.round(df_features, 2)

array([[0.  , 0.  , 0.  , ..., 1.  , 1.  , 0.04],
       [0.  , 0.  , 0.  , ..., 0.86, 0.94, 0.06],
       [0.  , 0.  , 1.  , ..., 0.86, 0.94, 0.11],
       ...,
       [0.  , 1.  , 0.  , ..., 0.  , 0.89, 0.11],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.07],
       [0.  , 0.  , 0.  , ..., 0.  , 0.44, 0.19]])

In [66]:
from sklearn.neighbors import NearestNeighbors

In [79]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(df_features)

In [80]:
distances, indices = nbrs.kneighbors(df_features)
prediction = []
found_rank = get_index_from_name('Move to Heaven')
for rank in indices[found_rank][1:]:
    prediction.append(df.iloc[rank]['judul'])

In [70]:
def get_index_from_name(judul):
  search_result = df[df['judul']==judul].index.tolist()
  if not search_result:
    return -1
  return search_result[0]

all_drama_names = list(df.judul.values)

def get_id_from_partial_name(partial):
  for judul in all_drama_names:
    if partial in judul:
      print(judul,all_drama_names.index(judul))

def print_similar_drama(query=None, rank=None):
  if rank:
    for rank in indices[rank][1:]:
      print(df.iloc[rank]['judul'])
  if query:
    found_rank = get_index_from_name(query)
    for rank in indices[found_rank][1:]:
      print(df.iloc[rank]['judul'])
    
def get_name(judul):
    df = df[df['judul'] == judul]    
    df_feature = pd.concat([df['genre'].str.strip().str.get_dummies(sep=','), df[['rating']],df[['tahun']],df[['episode']]],axis=1)
    return df_feature

In [71]:
import joblib
from sklearn.pipeline import make_pipeline

In [81]:
# pipeline = make_pipeline(MinMaxScaler(), nbrs)
# pipeline.fit(df_features)

joblib.dump(nbrs, '../data/output/redrakor.joblib')

['../data/output/redrakor.joblib']

In [75]:
nbrs

NearestNeighbors(algorithm='ball_tree', n_neighbors=6)